In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings


from src import conf

c:\Users\manuelalberto.romero\Documents\repos\dslabs\dslab-rag-e2e\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
QDRANT_API_KEY = os.environ["QDRANT_API_KEY"]

In [4]:
conf_settings = conf.load(file="settings.yaml")

LLM_WORKHORSE = conf_settings.llm_workhorse
LLM_FLAGSHIP = conf_settings.llm_flagship
EMBEDDINGS = conf_settings.embeddings
EMB_DIM = conf_settings.embeddings_dim

In [5]:
conf_infra = conf.load(file="infra.yaml")

VDB_URL = conf_infra.vdb_url

In [ ]:
from openai import OpenAI


client = OpenAI(
    api_key=MISTRAL_API_KEY,
    base_url="https://api.mistral.ai/v1/",
)

try:
    chat_completion = client.chat.completions.create(
        model="mistral-small-latest",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": "What is the capital of France?"},
        ],
    )

    print(chat_completion.choices[0].message.content)

except Exception as e:
    print(f"An error occurred: {e}")

The capital of France is Paris.


![MMR](docs/mmr.png)


[Qdrant MMR](https://qdrant.tech/blog/qdrant-1.15.x/#mmr-reranking)

In [7]:
client_qdrant = QdrantClient(
    api_key=QDRANT_API_KEY,
    url=VDB_URL
    )

try:
    client_qdrant.get_collections()
except Exception as err:
    print(err)

embeddings = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model=EMBEDDINGS
    )

try:
    embeddings.embed_query("abc")
except Exception as err:
    print(err)




if client_qdrant.collection_exists("space"):
    vector_store = QdrantVectorStore.from_existing_collection(
        embedding=embeddings,
        collection_name="space",
        url=VDB_URL,
        api_key=QDRANT_API_KEY
    )

In [12]:
retriever= vector_store.as_retriever(search_type="mmr")

docs = retriever.invoke("what did the president say about ketanji brown jackson?")
for i, doc in enumerate(docs):
    print(i)
    print(doc)
    print()

0
page_content='## 6.3 Sistemas de anillos y lunas notables  
Tanto Zephyros IV como Krion V presentan estructuras de anillos visibles, aunque los de Krion son más delgados y oscuros, compuestos mayoritariamente por polvo carbonáceo. Las lunas interiores de Zephyros interactúan gravitacionalmente con el sistema de anillos, generando ondas espirales similares a las observadas en Saturno.' metadata={'Header 1': '6. Planetas exteriores', 'Header 2': '6.3 Sistemas de anillos y lunas notables', '_id': '3874bf9d-d16d-423e-a0b4-95cd933a3462', '_collection_name': 'space'}

1
page_content='## 2.2 Disposición y dinámica de un sistema binario' metadata={'Header 1': '2. Conoce a Althéra', 'Header 2': '2.2 Disposición y dinámica de un sistema binario', '_id': 'd4bf1036-0a1c-4683-bdfe-b1d663f5ef6a', '_collection_name': 'space'}

2
page_content='## Anexo: Miscelánea
image_content: & Bellatrix\n\n*Betalgeuse\n\nCampoin 13 oct-2034\n- Oscilaciones periódicas\ndobles\n- Posible sistema binario\n- Explan

In [11]:
res = vector_store.similarity_search("what did the president say about ketanji brown jackson?")

for i, doc in enumerate(res):
    print(i)
    print(doc)
    print()

0
page_content='## 6.3 Sistemas de anillos y lunas notables  
Tanto Zephyros IV como Krion V presentan estructuras de anillos visibles, aunque los de Krion son más delgados y oscuros, compuestos mayoritariamente por polvo carbonáceo. Las lunas interiores de Zephyros interactúan gravitacionalmente con el sistema de anillos, generando ondas espirales similares a las observadas en Saturno.' metadata={'Header 1': '6. Planetas exteriores', 'Header 2': '6.3 Sistemas de anillos y lunas notables', '_id': '3874bf9d-d16d-423e-a0b4-95cd933a3462', '_collection_name': 'space'}

1
page_content='## Anexo: Miscelánea
image_content: & Bellatrix\n\n*Betalgeuse\n\nCampoin 13 oct-2034\n- Oscilaciones periódicas\ndobles\n- Posible sistema binario\n- Explanadas?\n× Rigel\n× Suph\n× Rigel
Figure 2 Notas de Mariela Estay el día del descubrimiento  
<!-- image -->' metadata={'Header 1': '10. Conclusiones y perspectivas futuras', 'Header 2': 'Anexo: Miscelánea', '_id': '9a419c71-c5f7-4575-8ee9-b461d9394ddc', '_

[Balancing Relevance and Diversity with MMR Search](https://qdrant.tech/blog/mmr-diversity-aware-reranking/)